In [13]:
import torch
from torch import nn
from d2l import torch as d2l

In [14]:
def pool2d(X,pool_size,mode='max'):
    p_h,p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1,X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[0]):
            if mode == 'max':
                Y[i,j] = X[i:i + p_h,j:j + p_w].max()
                #X[i:i+p_h, j:j+p_w]：截取输入 X 中，以 (i,j) 为左上角、大小为 p_h x p_w 的区域（池化核覆盖区域）
                #.max() / .mean()：对覆盖区域计算最大值 / 平均值，作为输出张量对应位置的元素。
            elif mode == 'avg':
                Y[i,j] = X[i:i + p_h,j:j + p_w].mean()
    return Y

In [15]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
pool2d(X,(2,2))

tensor([[4., 5.],
        [7., 8.]])

In [16]:
pool2d(X,(2,2),'avg')

tensor([[2., 3.],
        [5., 6.]])

In [17]:
#填充和步幅
X = torch.arange(16,dtype=torch.float32).reshape((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [18]:
#在默认情况下，深度学习框架中的不服和汇聚窗口的大小相同
pool2d = nn.MaxPool2d(3)
#nn.MaxPool2d(3)：创建最大池化层实例，核心参数说明：
# 3：池化核（汇聚窗口）大小为 3x3（仅传入整数时，默认是正方形核）；
# 默认参数：stride=pool_size（步幅 = 池化核大小，此处 stride=3）、padding=0（无填充）、dilation=1（普通池化）
pool2d(X)

tensor([[[[10.]]]])

In [19]:
#填充和步幅可以手动设定
pool2d = nn.MaxPool2d(3,padding=1,stride=2)
#padding=1：在输入特征图的边缘填充 1 层 0（上下左右各补 1 行 / 列），
# 目的是避免边缘特征丢失，同时可调整输出尺寸；

#第 1 次滑动（i=0, j=0）：
# 覆盖区域左上角坐标 (0,0)，右下角坐标 (2,2)，最大值为 5.0。
# 第 2 次滑动（i=0, j=2）：
# 向右移动 2 个像素，覆盖 (0,2) 到 (2,4)，最大值为 10.0。
# 第 3 次滑动（i=2, j=0）：
# 向下移动 2 个像素，覆盖 (2,0) 到 (4,2)，最大值为 13.0。
# 第 4 次滑动（i=2, j=2）：
# 向下、向右各移动 2 个像素，覆盖 (2,2) 到 (4,4)，最大值为 14.0。
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [20]:
#可以设定一个任意大小的矩形汇聚窗口，分别设定填充和步幅的高度和宽度
pool2d = nn.MaxPool2d((2,3),stride = (2,3),padding=(0,1))
#stride=(2, 3)：步幅为矩形，高度方向步幅 = 2、宽度方向步幅 = 3；
#padding=(0, 1)：填充为矩形，高度方向不填充（0）、宽度方向填充 1 层（左右各补 1 列）。
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [21]:
#6.5多个通道,汇聚层在每个输入通道上单独运算，这意味着汇聚层的输出通道和输入通道相同
X = torch.cat((X,X+1),1)
#torch.cat(...)：将两个张量在指定维度拼接，实现通道数扩展：
# 第一个参数 (X, X+1)：待拼接的两个张量，其中 X+1 是将原 X 的每个元素加 1（生成第二个通道的数据）。
# 第二个参数 1：拼接维度为通道维度（PyTorch 中通道维度索引为 1）
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

In [22]:
pool2d = nn.MaxPool2d(3,padding=1,stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])